### Feature 6 - Usage of Gas Tokens

In [2]:
from web3 import Web3
from tqdm import tqdm
import pandas as pd
import dotenv
import os

In [3]:
%load_ext dotenv
%dotenv

In [4]:
node_url=os.getenv("NODE_URL")
web3 = Web3(Web3.HTTPProvider(node_url))
print(web3.is_connected())

True


In [33]:
web3.eth.get_transaction('0x10504c0666a22466911d038d2b1c895b7ea5ec05cef107e46d3cae0121cfcda7')


In [78]:
web3.eth.get_transaction('0x8abbef4de9993df03fcba73f80d0d245939836fbf75265d80df84b852ce39453')

#### Naive approach with web3 - Loop through AccessList

In [53]:
gas_token_addresses = {"0x0000000000b3F879cb30FE243b4Dfee438691c04": "GST2",
                       "0x88d60255F917e3eb94eaE199d827DAd837fac4cB": "GST1",
                       "0x0000000000004946c0e9F43F4Dee607b0eF1fA1c": "CHI"}

In [6]:
block_number = 19587005
block = web3.eth.get_block(block_number, full_transactions=True)

for transaction in block.transactions:  
    
    if (transaction["from"] == "0x678111a6cA5749f1744b5E080A855CEC8d631E20"):
        
        if "accessList" not in transaction:
            print("AccessList not in Transaction")
        
        for element in transaction["accessList"]:
            if element["address"] in gas_token_addresses.values():
                print(element["address"])


In [7]:
def is_transaction_using_gas_token(block_number, address, web3):
    block = web3.eth.get_block(block_number, full_transactions=True)
    
    for transaction in block.transactions:   
        
        if not transaction["from"] == address:
            continue
    
        if "accessList" not in transaction:
            #print(transaction)
            continue
            
        for element in transaction["accessList"]:
            if element["address"] in gas_token_addresses.values():
                print(element["address"])
                return True
    return False

In [22]:
df_insertion = pd.read_csv ('../data/insertion_attacks.csv', delimiter=',')


In [9]:
for index, entry in tqdm(df_insertion.sample(1000).iterrows()):
        block_nr = entry["Block Number"]
        address = entry["First Attacker"]
        
        is_transaction_using_gas_token(block_nr, address, web3)

#### Approach using web3 API - Recursive scanning of AttributeDict

In [10]:
import web3.datastructures as wd

def check_value_of_interest(data, transactions_with_value_of_interest, hash):
    
    if isinstance(data, wd.AttributeDict):
        for key, value in data.items():
            if value in gas_token_addresses.values():
                gas_token = list(filter(lambda x: gas_token_addresses[x] == value, gas_token_addresses))[0]
                transactions_with_value_of_interest.append((data, gas_token))
                print(f"Gas Token used in {data}: {gas_token} -> {value}")
                break  
            check_value_of_interest(value, transactions_with_value_of_interest, hash)
    
    elif isinstance(data, list):
        for item in data:
            check_value_of_interest(item, transactions_with_value_of_interest, hash)

# List to store transactions with the value of interest
transactions_with_value_of_interest = []

transactions = web3.eth.get_transaction('0xdcdc49f7bf2d1c97134df53de8090aef064b4611c17490020ace706cffecd35e')

# Call the recursive function for each AttributeDict object
for transaction_data in [transactions]:  # assuming you have two AttributeDict objects
    check_value_of_interest(transaction_data, transactions_with_value_of_interest, '0xdcdc49f7bf2d1c97134df53de8090aef064b4611c17490020ace706cffecd35e')

# Print the transactions with the value of interest
for transaction in transactions_with_value_of_interest:
    print(transaction)


In [23]:

def is_transaction_using_gas_token(block_number, address, web3):
    block = web3.eth.get_block(block_number, full_transactions=True)
    
    transactions_with_value_of_interest = []
    
    for transaction in block.transactions:  
        
        if not (transaction["from"] == address or transaction["to"] == address):
            continue
        else:
            hash = transaction["hash"].hex()
            print(f"Transaction: {hash}\n")
            print(transaction)
            print("\n")

        
        check_value_of_interest(transaction, transactions_with_value_of_interest, transaction["hash"])
        

In [24]:
for index, entry in tqdm(df_insertion.sample(10).iterrows()):
        block_nr = entry["Block Number"]
        address = entry["First Attacker"]
        
        is_transaction_using_gas_token(block_nr, address, web3)

#### Approach Etherscan Parsing

In [61]:
from bs4 import BeautifulSoup
import re

In [59]:
def parse_etherscan_for_transaction(transaction_hash):
    
    base_url = 'https://etherscan.io/tx/'
    url = base_url + transaction_hash
    
    # To Beat 403-Error
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    # Fetch 
    response = requests.get(url, headers=headers)    
    
    has_gas_token_flag = False

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        html_text = BeautifulSoup(response.content, 'html.parser')
        
        # Convert the soup object to string
        html_string = str(html_text)
        
        # Perform regex search on the HTML string for each word
        for word in gas_token_addresses.keys():
            matches = re.findall(r'\b' + re.escape(word) + r'\b', html_string, flags=re.IGNORECASE)
            
            # Print the matches
            if matches:
                print(f"'{gas_token_addresses[word]}' : {word} found!")
                has_gas_token_flag = True
    
    else:
        print("Could not Parse Etherscan")
        
    return has_gas_token_flag

In [62]:
transaction_with_chi_gastoken = "0x10504c0666a22466911d038d2b1c895b7ea5ec05cef107e46d3cae0121cfcda7"
parse_etherscan_for_transaction(transaction_with_chi_gastoken)

In [67]:
def is_transaction_using_gas_token(block_number, address, web3):
    
    block = web3.eth.get_block(block_number, full_transactions=True)
    
    for transaction in block.transactions:  
        
        if not (transaction["from"] == address or transaction["to"] == address):
            continue
        else:
            hash = transaction["hash"].hex()
            print(f"Transaction: {hash}\n")
            parse_etherscan_for_transaction(hash)
            print("\n")
            break

In [79]:
counter = 1
for index, entry in df_insertion.sample(10).iterrows():
    block_nr = entry["Block Number"]
    address = entry["First Attacker"]
    print(address, counter)
    counter += 1
    is_transaction_using_gas_token(block_nr, address, web3)

#### Approach Etherscan API - Loop through internal transactions

In [82]:
import requests

def get_internal_transactions(tx_hash):
    # API endpoint
    url = 'https://api.etherscan.io/api'

    # Parameters
    params = {
        'module': 'account',
        'action': 'txlistinternal',
        'txhash': tx_hash,
        'apikey': '1PN1111XBM2W5HIQCSMQH6RA65JVYPQM1R'
    }

    try:
        # Sending GET request
        response = requests.get(url, params=params, timeout=3)
    
        # Checking if request was successful
        if response.status_code == 200:
            data = response.json()
            return data["result"]
        else:
            #print('Error occurred:', response.status_code)
            return None
        
    except requests.exceptions.Timeout:
        #print('Request did not go through: timeout occurred')
        return None

In [83]:
def is_gas_token_contract_in_internal_transaction(transaction_hash):
    
    gas_token_addresses = {"0x0000000000b3f879cb30fe243b4dfee438691c04": "GST2",
                           "0x88d60255f917e3eb94eae199d827dad837fac4cb": "GST1",
                           "0x0000000000004946c0e9f43f4dee607b0ef1fa1c": "CHI"}
    
    internal_transactions = get_internal_transactions(transaction_hash)
    
    if not internal_transactions:
        return False
    
    for transaction in internal_transactions:
        if transaction["from"] in gas_token_addresses.keys():
            #print(f"'{gas_token_addresses[transaction['from']]}' is used in {transaction['type']}")
            return True
        if transaction["to"] in gas_token_addresses.keys():
            #print(f"'{gas_token_addresses[transaction['to']]}' is used in {transaction['type']}")
            return True
    
    return False
    

In [84]:
def is_transaction_using_gas_token(block_number, address, web3):
    
    block = web3.eth.get_block(block_number, full_transactions=True)
    
    for transaction in block.transactions:  
        
        if not (transaction["from"] == address or transaction["to"] == address):
            continue
            
        else:
            transaction_hash = transaction["hash"].hex()
            #print(f"Transaction: {transaction_hash}\n")
            return is_gas_token_contract_in_internal_transaction(transaction_hash)

##### Override usedGasToken in already sampled data

In [92]:
def get_gas_token(file_name):
    
    df = pd.read_csv(f'../data/{file_name}', delimiter=',')

    entries = []
    
    for index, entry in tqdm(df.iterrows()):
        block_nr = entry["blockNumber"]
        address = entry["address"]
        entry["usedGasToken"] = is_transaction_using_gas_token(block_nr, address, web3)
        #print(address, counter)
        #counter += 1
        entries.append(entry)
    
    return pd.DataFrame(entries)

In [93]:
insertion_first_atk__with_gas_token_df = get_gas_token("insertion_atks_first_atk_2500.csv")
insertion_first_atk__with_gas_token_df.to_csv('../data/insertion_atks_first_atk_with_gas_token.csv')

In [94]:
insertion_second_atk__with_gas_token_df = get_gas_token("insertion_atks_second_atk_2500.csv")
insertion_second_atk__with_gas_token_df.to_csv('../data/insertion_atks_second_atk_with_gas_token.csv')

In [95]:
random_sampled_transactions_with_gas_token_df = get_gas_token("random_sampled_transactions.csv")
random_sampled_transactions_with_gas_token_df.to_csv('../data/random_sampled_transactions_with_gas_token.csv')

Distribution of usedGasToken in attack data vs. randomly sampled data

In [103]:
df_first_atks = pd.read_csv(f'../data/insertion_atks_first_atk_with_gas_token.csv', delimiter=',')
df_second_atks = pd.read_csv(f'../data/insertion_atks_second_atk_with_gas_token.csv', delimiter=',')
df_first_atks.usedGasToken.value_counts() + df_second_atks.usedGasToken.value_counts()


In [104]:
df_random = pd.read_csv(f'../data/random_sampled_transactions_with_gas_token.csv', delimiter=',')
df_random.usedGasToken.value_counts()